In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import duckdb
import os
import glob

In [2]:
def read_zipfile(name):
    archive_names = ['uid', 'utc', 'open', 'close', 'high', 'low', 'volume', 'other']
    with ZipFile(name) as f:
        df = pd.concat([
            pd.read_csv(f.open(csv_name), sep = ';', header = None, names = archive_names) for csv_name in sorted(f.namelist())
        ], axis = 0)
    return df

In [3]:
# sorted(glob.glob("../data/archive/*.zip"))

In [4]:
df = pd.concat([read_zipfile(zip_name) for zip_name in sorted(glob.glob("../data/archive/BBG004730RP0_*.zip"))],axis = 0)

In [5]:
df.head()

,uid,utc,open,close,high,low,volume,other
0,962e2a95-02a9-4171-abd7-aa198dbe643a,2018-03-07T18:33:00Z,138.43,138.40,138.45,138.40,498,NaN
1,962e2a95-02a9-4171-abd7-aa198dbe643a,2018-03-07T18:34:00Z,138.45,138.48,138.48,138.41,733,NaN
2,962e2a95-02a9-4171-abd7-aa198dbe643a,2018-03-07T18:35:00Z,138.42,138.44,138.48,138.40,1262,NaN
3,962e2a95-02a9-4171-abd7-aa198dbe643a,2018-03-07T18:36:00Z,138.38,138.36,138.41,138.36,893,NaN
4,962e2a95-02a9-4171-abd7-aa198dbe643a,2018-03-07T18:37:00Z,138.36,138.40,138.43,138.36,1870,NaN


## add to DB

### archive history

In [22]:
def load_1min_history(db_name = "tinvest.duckdb", history_path = "../data/archive/", table_name = 'candles_1min'):
    create_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        uid UUID,
        utc TIMESTAMP,
        open FLOAT,
        close FLOAT,
        high FLOAT,
        low FLOAT,
        volume UINTEGER,
        PRIMARY KEY (uid, utc))
    """
    with duckdb.connect(db_name) as conn:
        conn.sql(create_query)
        for zip_name in sorted(glob.glob(history_path + "*.zip")):
            print(f"READING {zip_name}...", end = '')
            candles_df = read_zipfile(zip_name)
            del candles_df['other']
            conn.sql("INSERT INTO candles_1min SELECT * FROM candles_df")
            print("ADDED TO", table_name)

In [23]:
load_1min_history()

READING ../data/archive/BBG000F6YPH8_2019.zip...ADDED TO candles_1min
READING ../data/archive/BBG000F6YPH8_2020.zip...ADDED TO candles_1min
READING ../data/archive/BBG000F6YPH8_2021.zip...ADDED TO candles_1min
READING ../data/archive/BBG000F6YPH8_2022.zip...ADDED TO candles_1min
READING ../data/archive/BBG000F6YPH8_2023.zip...ADDED TO candles_1min
READING ../data/archive/BBG000F6YPH8_2024.zip...ADDED TO candles_1min
READING ../data/archive/BBG000LNHHJ9_2018.zip...ADDED TO candles_1min
READING ../data/archive/BBG000LNHHJ9_2019.zip...ADDED TO candles_1min
READING ../data/archive/BBG000LNHHJ9_2020.zip...ADDED TO candles_1min
READING ../data/archive/BBG000LNHHJ9_2021.zip...ADDED TO candles_1min
READING ../data/archive/BBG000LNHHJ9_2022.zip...ADDED TO candles_1min
READING ../data/archive/BBG000LNHHJ9_2023.zip...ADDED TO candles_1min
READING ../data/archive/BBG000LNHHJ9_2024.zip...ADDED TO candles_1min
READING ../data/archive/BBG000NLCCM3_2018.zip...ADDED TO candles_1min
READING ../data/arch

### metainfo

In [35]:
shares_info_df = pd.read_csv('../data/shares_info.csv', parse_dates=['ipo_date', 'first_1min_candle_date', 'first_1day_candle_date'])

In [39]:
shares_info_df.head()

,figi,ticker,class_code,isin,lot,currency,short_enabled_flag,name,exchange,ipo_date,...,position_uid,asset_uid,instrument_exchange,for_iis_flag,for_qual_investor_flag,weekend_flag,blocked_tca_flag,liquidity_flag,first_1min_candle_date,first_1day_candle_date
0,TCS10A0JKQU8,RU000A0JKQU8,TILA21,RU000A0JKQU8,1,rub,True,Магнит,unknown,2004-03-04,...,0149f24b-d4c8-4360-accc-9a7ec07eb1af,4833e124-265f-4879-adc8-58bdf983f54e,0,False,False,False,False,True,1970-01-01,1970-01-01
1,US9100471096,US9100471096,SPBXM_OTC,US9100471096,1,usd,True,United Airlines Holdings,unknown,1970-01-01,...,6bbde3a6-1f12-457c-a265-b19b5e9d871f,6a802865-717d-4389-823f-1bb5baaac2bf,0,False,True,False,True,False,1970-01-01,1970-01-01
2,LU2314763264,LU2314763264,SPBXM_OTC,LU2314763264,1,usd,False,Arrival,unknown,1970-01-01,...,54a1efa4-e124-4bd7-afa3-dca5ef84207a,9e644a19-dc66-47e5-8251-33a0dcd7e3b0,0,False,True,False,True,True,1970-01-01,1970-01-01
3,TCS9B6T5S470,JE00B6T5S470,00,JE00B6T5S470,1,rub,False,Solidcore,unknown,2011-10-28,...,b049e81f-2882-4231-86f4-ea0e9d768ea1,afbfbb30-28e4-477a-a8aa-3eb983d0d2e4,0,True,False,False,False,True,1970-01-01,1970-01-01
4,TCSF766T1007,US00766T1007,A30,US00766T1007,1,usd,False,AECOM,unknown,2014-07-31,...,74991582-8d8e-412c-886e-39ad26a16d49,eb89d4a2-0edb-4f3b-9e24-4ce8d2f0fe28,0,False,True,False,True,False,1970-01-01,1970-01-01


In [38]:
shares_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12072 entries, 0 to 12071
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   figi                      12072 non-null  object        
 1   ticker                    12072 non-null  object        
 2   class_code                12072 non-null  object        
 3   isin                      12072 non-null  object        
 4   lot                       12072 non-null  int64         
 5   currency                  12072 non-null  object        
 6   short_enabled_flag        12072 non-null  bool          
 7   name                      12072 non-null  object        
 8   exchange                  12072 non-null  object        
 9   ipo_date                  12072 non-null  datetime64[ns]
 10  issue_size                12072 non-null  int64         
 11  country_of_risk           11809 non-null  object        
 12  country_of_risk_na

In [37]:
create_shares_info = "CREATE TABLE IF NOT EXISTS shares_info AS SELECT * FROM shares_info_df"
with duckdb.connect('../data/tinvest.duckdb') as conn:
    conn.sql(create_shares_info)

### test

In [7]:
with duckdb.connect("../data/tinvest.duckdb") as conn:
    df = conn.sql("""
    SELECT  i.figi, i.ticker, c.utc AS dtime, 
    c.open, c.close, c.high, c.low, c.volume, i.lot, i.issue_size,
    i.currency, i.exchange, i.sector FROM shares_info i INNER JOIN candles_1min C ON i.uid = c.uid
    WHERE dtime > '2024-07-01'
    """).df()

In [11]:
# df.groupby('ticker')['dtime'].agg(['min', 'max', 'count'])

In [13]:
# df.query("ticker == 'VTBR'")['volume'].plot()